In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose


In [ ]:
data = pd.read_csv("times_series_data_no_labels.csv" ,
    index_col='datetime',
    parse_dates=['datetime']
    )
data.describe()

In [ ]:
ts = data['data_0'].asfreq('5min')
result = adfuller(ts)
if result[1] > 0.05:
    ts = ts.diff().dropna()

# Plot ACF and PACF
plot_acf(ts)
plot_pacf(ts)
plt.show()


In [ ]:
ts_test = ts[-11840:]
ts = ts[:-11840]

In [ ]:
# Fit SARIMA model
p, d, q = 2, 0, 0
P, D, Q, s = 1, 1, 0, 288  # s = 288 for daily seasonality in 5-minute intervals


ts = ts[:-1000]
model = SARIMAX(ts, order=(p, d, q), seasonal_order=(P, D, Q, s))
results = model.fit(disp=True)
print(results.summary())

# Diagnostics

results.plot_diagnostics(figsize=(15, 12))
plt.show()

# Forecast
forecast = results.get_forecast(steps=11840)
forecast_ci = forecast.conf_int()